In [2]:
# For easier dev of local modules:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:

# External Dependencies:
import boto3
import sagemaker
from sagemaker.pytorch.estimator import PyTorch as PyTorchEstimator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
BUCKET_NAME = "2020-05-gym-bert"
%store BUCKET_NAME

Stored 'BUCKET_NAME' (str)


In [5]:
role = sagemaker.get_execution_role()
botosess = boto3.session.Session()
region = botosess.region_name
s3 = botosess.resource("s3")
bucket = s3.Bucket(BUCKET_NAME)
smclient = botosess.client("sagemaker")

In [7]:
!pip3 install torchnlp

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import csv
from torchnlp.datasets import imdb_dataset
train, test = imdb_dataset(train=True,test=True)

with open('data/train.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['text','sentiment'])
    for i in train:
        csvwriter.writerow([i['text'],i['sentiment']])

with open('data/test.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['text','sentiment'])
    for i in test:
        csvwriter.writerow([i['text'],i['sentiment']])

ModuleNotFoundError: No module named 'torchnlp.datasets'

In [ ]:
train_filename = 'SQuAD_train-v1.1.json'
val_filename = 'SQuAD_dev-v1.1.json'

In [ ]:
boto3.Session().resource('s3').Bucket(BUCKET_NAME).Object(train_filename).upload_file('./dataset/train-v1.1.json')
boto3.Session().resource('s3').Bucket(BUCKET_NAME).Object(val_filename).upload_file('./dataset/dev-v1.1.json')

In [ ]:
train_channel = f"s3://{BUCKET_NAME}/{train_filename}"

In [ ]:
val_channel = f"s3://{BUCKET_NAME}/{val_filename}"

In [ ]:
estimator = PyTorchEstimator(
    entry_point="train.py",
    source_dir="src",

    base_job_name="bert-qna",
    #checkpoint_s3_uri=f"s3://{BUCKET_NAME}/checkpoints",
    output_path=f"s3://{BUCKET_NAME}/",

    framework_version="1.4.0",
    py_version="py3",

    role=role,
    train_instance_count=1,
    train_instance_type="local", # "ml.p2.xlarge",
    train_max_run=60*60,
    train_max_wait=60*60,
    # Checkpoint saving might be part-working but resume definitely isn't yet:
    #train_use_spot_instances=True,

    hyperparameters={
        "seed": 1337,
        #"log-level": "DEBUG",
    },
)

In [ ]:
estimator.fit({ "train": train_channel, "validation": val_channel })